In [10]:
import numpy as np
import pandas as pd
import matplotlib
import cv2
import torchvision
import torch
from torch.autograd import Variable
from torch import Tensor, cat, sigmoid, exp, stack, max, meshgrid, linspace, arange
from model import Darknet, Tail, Head, Sequential, Yolo

#from torch.nn import Conv2d
#from weight_formater import darknet, tail, we#ight, weights

In [11]:
anchors = Tensor([[[22, 59],  [61, 43],  [50, 97]],
                  [[22, 59],  [61, 43],  [50, 97]],
                  [[12, 25],  [23, 16],  [36, 26]]])
darknet = Darknet()
yolo = Yolo(anchors,[3,3,3], 1)

tensor([[22., 59.],
        [61., 43.],
        [50., 97.]])


In [12]:
len(anchors[0].view(3,1,1,2))

3

In [13]:
yolo.eval()

Yolo(
  (feature_extractor): Darknet(
    (intro): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
    (module_list): ModuleList(
      (0): ModuleList(
        (0): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.1)
        )
        (1): Sequential(
          (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.1)
          (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [14]:
with open( "/home/ivan/sets/best_big/backup/v3single_last.weights", "rb") as f:
             header = np.fromfile(f, dtype=np.int32, count=5)
             seen = header[3]
             flatten_weights = np.fromfile(f, dtype=np.float32)

In [15]:
yolo.load_weights(flatten_weights)

In [16]:
number_of_classes = 1

In [17]:
image = cv2.imread("/home/ivan/Desktop/BN-UQ280_FEDUCI_GR_20170810142213.jpg")
image = cv2.resize(image, (416, 416), interpolation=cv2.INTER_LINEAR)
img_ =  image[:,:,::-1].transpose((2,0,1))  # BGR -> RGB | H X W C -> C X H X W 
img_ = img_[np.newaxis,:,:,:]/255.0  
image_tensor = img_     
image_tensor = Variable(torch.from_numpy(image_tensor).float())
#image_tensor = image_tensor.permute(2, 0, 1)

In [18]:
head = Head(Tensor([[120, 93],  [84,191], [238,186]]),1)
anchors=Tensor([[120, 93],  [84,191], [238,186]]).view(3, 1, 1, 2)
head_1 = Head(Tensor([[22, 59],  [61, 43],  [50, 97]]),1)
anchors_1=Tensor([[22, 59],  [61, 43],  [50, 97]]).view(3, 1, 1, 2)
head_2 = Head(Tensor([[12, 25],  [23, 16],  [36, 26]]),1)
anchors_2 = Tensor([[12, 25],  [23, 16],  [36, 26]]).view(3, 1, 1, 2)

In [20]:
image_tensor.size()

torch.Size([1, 3, 416, 416])

In [19]:
detections = yolo(image_tensor)

torch.Size([1, 256, 52, 52])
torch.Size([1, 512, 26, 26])
torch.Size([1, 1024, 13, 13])


RuntimeError: shape '[-1, 3, 6, 52, 52]' is invalid for input of size 692224

In [21]:
3*6*52*52

48672

In [ ]:
dark = tail(darknet(image_tensor))

In [ ]:
features = dark[0]
features_1 = dark[1]
features_2 = dark[2]

In [ ]:
centers, sizes, probabilities = head(features)

In [ ]:
stack((cat((centers, sizes),-1),cat((centers, sizes),-1)))

In [ ]:
Tensor([[12, 25],  [23, 16],  [36, 26]])[0]

In [ ]:
torchvision.ops.nms(boxes, scores, iou_threshold)

In [ ]:
boxes = pd.DataFrame(centers.view(-1,2).detach().numpy() * 416, columns=["center_x","center_y"])
boxes["width"]=0
boxes['height']=0
boxes["obj"] = probabilities.view(-1,2)[...,0].detach().numpy()
boxes[["width","height"]] = sizes.view(-1,2).detach().numpy()
boxes["obj"].max()

boxes_1 = pd.DataFrame(centers_1.view(-1,2).detach().numpy() * 416, columns=["center_x","center_y"])
boxes_1["width"]=0
boxes_1['height']=0
boxes_1["obj"] = probabilities_1.view(-1,2)[...,0].detach().numpy()
boxes_1[["width","height"]] = sizes_1.view(-1,2).detach().numpy()
boxes_1["obj"].max()

boxes_2 = pd.DataFrame(centers_2.view(-1,2).detach().numpy() * 416, columns=["center_x","center_y"])
boxes_2["width"]=0
boxes_2['height']=0
boxes_2["obj"] = probabilities_2.view(-1,2)[...,0].detach().numpy()
boxes_2[["width","height"]] = sizes_2.view(-1,2).detach().numpy()
boxes_2["obj"].max()
#boxes =boxes.append(boxes_1).append(boxes_2)

In [ ]:
from src.boxes_manipulations import convert_to_matplotlib
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1)
for x,y, w, h in boxes.loc[boxes["obj"]>0.1,["center_x", "center_y","width", "height"]].values:
       #ax.plot([x],[y],"r+", linewidth=9)
       ax.add_patch(plt.Rectangle((int(x-w/2), int(y-h/2)), int(w), int(h), linewidth=4, fill=False))
plt.imshow(image)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1)
for x,y, w, h in boxes_1.loc[boxes_1["obj"]>0.,["center_x", "center_y","width", "height"]].values:
       #ax.plot([x],[y],"r+", linewidth=9)
       ax.add_patch(plt.Rectangle((int(x-w/2), int(y-h/2)), int(w), int(h), linewidth=4, fill=False))
plt.imshow(image)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1)
for x,y, w, h in boxes_2.loc[boxes_2["obj"]>0.9,["center_x", "center_y","width", "height"]].values:
       #ax.plot([x],[y],"r+", linewidth=9)
       ax.add_patch(plt.Rectangle((int(x-w/2), int(y-h/2)), int(w), int(h), linewidth=4, fill=False))
plt.imshow(image)
plt.show()

In [ ]:
boxes =boxes.append(boxes_1).append(boxes_2)
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1)
for x,y, w, h in boxes.loc[boxes["obj"]>0.1,["center_x", "center_y","width", "height"]].values:
       #ax.plot([x],[y],"r+", linewidth=9)
       ax.add_patch(plt.Rectangle((int(x-w/2), int(y-h/2)), int(w), int(h), linewidth=4, fill=False))
plt.imshow(image)
plt.show()